<a href="https://colab.research.google.com/github/anwar-ai/PIFE-/blob/master/PIFE_ENSAM_IPP_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Texte alternatif…](https://cdn-03.9rayti.com/rsrc/cache/widen_750/uploads/2011/05/ENSAM-Meknes-Journ%C3%A9e-%C2%ABPortes-ouvertes-%C2%BB-2011.jpg)
# **Automatisation des procédés CND par l'intelligence artificielle**
  Les rayons X et gamma en contrôle non destructif sont utilisés pour réaliser des radiographies afin de caractériser l'état d'intérgrité des structures et des matériaux sans les dégrader, la radiographie est utilsé notament pour les soudures dans les centrales nucléaires, dans le secteur pétrolier, pour la corrosion des tuyaux ou les fissures des pièces mécaniques. 
Dans industrie automobile l'inspection des pièces se fait avec échantionnage et d'une manière unitaire, ce qui affecte le temps de production et le taux de qualité (main d'oeuvre qualifié). Le but de cette étude est d'utiliser l'appretissage supervisé profond (Deep Learning) pour la détéction automatique des retassures à lintérieur des jantes automobile fabriquées en AlSi moulé.


Anwar Aachati
anwar.aachati@altran.com

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import cv2
import os
from PIL import Image
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential

In [ ]:
!ls

In [ ]:
!git clone https://github.com/anwar-ai/PIFE-.git

In [ ]:
!ls

In [ ]:
x=[]
l=[i for i in os.listdir("PIFE-/img")]
l.sort()
for img in l: 
  image=cv2.imread("PIFE-/img/"+img,0)
  image=cv2.resize(image,(512,512), interpolation = cv2.INTER_AREA)
  x.append(image)
x_train=np.array(x).reshape(44,512,512,1) 
  

In [ ]:
plt.figure(figsize=(17,9))
plt.imshow(x_train[1,:,:,0],cmap="gray")
plt.show()

In [ ]:
len(x),x_train.shape

In [ ]:
y=[]
k=[i for i in os.listdir("PIFE-/mask")]
k.sort()
for img in k: 
  image=cv2.imread("PIFE-/mask/"+img,0)
  image=cv2.resize(image,(512,512), interpolation = cv2.INTER_AREA)
  y.append(image)
mask=np.array(y).reshape(44,512,512,1) 
plt.figure(figsize=(17,9))
plt.imshow(mask[1,:,:,0],cmap="gray")
plt.show()

In [ ]:
len(y),mask.shape


In [ ]:
import tensorflow as tf

mask=mask.astype('float32')
tf.config.experimental_run_functions_eagerly(True)


In [ ]:
lignes = 512
colonnes = 512

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from scipy.ndimage.measurements import label
import time

## **U-net convolutional neural network** 
![Texte alternatif…](https://miro.medium.com/max/2824/1*f7YOaE4TWubwaFF7Z1fzNw.png)
U-Net est une architecture  appelée «réseau entièrement convolutionnel» ,développé pour la segmentation d'images biomédicales au département d'informatique de l'université de Fribourg en Allemagne.

In [ ]:
def unet():
    inputs = Input((lignes, colonnes, 1))
    inputs_norm = Lambda(lambda x: x/127.5 - 1.)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(64, kernel_size=(
        2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(32, kernel_size=(
        2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(16, kernel_size=(
        2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(8, kernel_size=(
        2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [ ]:
model = unet()

# **Intersection over union "IoU"**
![Texte alternatif…](http://media5.datahacker.rs/2018/11/IoU.png)


c'est un indice de performace qui donne une idée sur la précision du modèle

In [ ]:
def smooth_dice_coeff(smooth=1.):

    smooth = float(smooth)

    def IOU_calc(y_true, y_pred):
            y_true_f = K.flatten(y_true)
            y_pred_f = K.flatten(y_pred)
            intersection = K.sum(y_true_f * y_pred_f)

            return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

    def IOU_calc_loss(y_true, y_pred):
        return -IOU_calc(y_true, y_pred)
    return IOU_calc, IOU_calc_loss

IOU_calc, IOU_calc_loss = smooth_dice_coeff(0.00001)

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss=IOU_calc_loss, metrics=[IOU_calc])

In [ ]:
history = model.fit(x_train, mask, batch_size=10, epochs=200, verbose=1,shuffle=True)

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(model.history.history['loss'], label='Train loss')

plt.xlabel('Epochs')
plt.ylabel('Erreur')
plt.legend()

In [ ]:
plt.figure(figsize=(6, 5))
plt.plot(model.history.history['IOU_calc'], label='Train IOU')

plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()

In [ ]:
x_test=x_train[:,:,:,:]
predict = model.predict(x_test)
x_test.shape

In [ ]:
import cv2


def predict_evaluation(pred, image, label):
    '''
    '''
    # transformer  l'image grise en rgb
    img = np.array(image, np.uint8)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # standariser l'image to 0~255
    im_label = np.array(255*label, dtype=np.uint8)
    im_pred = np.array(255*pred, dtype=np.uint8)

    # transformer tout en  rgb
    rgb_label = cv2.cvtColor(im_label, cv2.COLOR_GRAY2RGB)
    rgb_pred = cv2.cvtColor(im_pred, cv2.COLOR_GRAY2RGB)

  
    rgb_label[:, :, 0] = 0*rgb_label[:, :, 0]
    rgb_label[:, :, 1] = 0*rgb_label[:, :, 1]
    rgb_pred[:, :, 1] = 0*rgb_pred[:, :, 1]
    rgb_pred[:, :, 2] = 0*rgb_pred[:, :, 2]

    img_pred = cv2.addWeighted(rgb_img, 1, rgb_pred, 1, 0)
    img_label = cv2.addWeighted(rgb_img, 1, rgb_label, 1, 0)

# le resizing
    rgb_img=cv2.resize(rgb_img,(1080,900), interpolation = cv2.INTER_AREA)
    img_pred=cv2.resize(img_pred,(1080,900), interpolation = cv2.INTER_AREA)
    img_label=cv2.resize(img_label,(1080,900), interpolation = cv2.INTER_AREA)
    plt.figure(figsize=(10, 10))
 
#show_pyplot
    plt.figure(figsize=(20,12))
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_img)
    plt.title('Image Originale ')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(img_pred)
    plt.title('Prédiction')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(img_label)
    plt.title('Realité')
    plt.axis('off')

In [ ]:
predict_evaluation(predict[0,:,:,0], x_test[0,:,:,0], mask[0,:,:,0])

In [ ]:
predict_evaluation(predict[9,:,:,0], x_test[9,:,:,0], mask[9,:,:,0])

In [ ]:
predict_evaluation(predict[12,:,:,0], x_test[12,:,:,0], mask[12,:,:,0])

In [ ]:
predict_evaluation(predict[22,:,:,0], x_test[22,:,:,0], mask[22,:,:,0])

In [ ]:
predict_evaluation(predict[36,:,:,0], x_test[36,:,:,0], mask[36,:,:,0])

In [ ]:
predict_evaluation(predict[11,:,:,0], x_test[11,:,:,0], mask[11,:,:,0])



# **Construction des Bounding boxes "Bboxes"**








In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
def seuil (y):
  p=y
  for i in range (y.shape[0]):
    for j in range (y.shape[1]):
      if p[i][j]<100:
         p[i][j]=0
         
       
  return (p)

In [ ]:
def bbox(g):
    row=[]
    col=[] 
    for i in range (0,511):
        for j in range (0,511):
            if g[i][j]!=0:
                row.append(i)
                col.append(j)                
    xmax=max(row)
    xmin=min(row)
    ymax=max(col)
    ymin=min(col) 
    return((ymin,xmin),(ymax,xmax))

In [ ]:
def Draw_bbox(img,ori):
    img=seuil(img*255)
    window_name = 'bbx'
    start_point = bbox(img)[0] 
    end_point =bbox(img)[1] 
    color = (255, 0, 255) 
    thickness = 1
    image = cv2.putText(ori, 'Retassure', start_point, cv2.FONT_HERSHEY_SIMPLEX ,  
                   0.45, color, 1, cv2.LINE_AA) 
    image = cv2.rectangle(ori, start_point, end_point, color, thickness) 
    cv2_imshow( image) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()
   

In [ ]:
Draw_bbox(predict[1,:,:,0],x_test[1,:,:,0])

In [ ]:
Draw_bbox(predict[38,:,:,0],x_test[38,:,:,0])

In [ ]:
Draw_bbox(predict[11,:,:,0],x_test[11,:,:,0])

In [ ]:
Draw_bbox(predict[23,:,:,0],x_test[23,:,:,0])

In [ ]:
Draw_bbox(predict[3,:,:,0],x_test[3,:,:,0])